In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
import malaya_speech
import random
import numpy as np
import malaya
from sklearn.utils import shuffle
from sklearn.utils.random import sample_without_replacement

`pyaudio` is not available, `malaya_speech.streaming.pyaudio` is not able to use.


In [3]:
malaya_speech.force_alignment.transducer.available_transformer()

,Size (MB),Quantized Size (MB),Language
conformer-transducer,120,32.3,[malay]
conformer-transducer-mixed,120,32.3,"[malay, singlish]"
conformer-transducer-singlish,120,32.3,[singlish]


In [4]:
model = malaya_speech.force_alignment.transducer.transformer(model = 'conformer-transducer', device = 'gpu:0')
singlish_model = malaya_speech.force_alignment.transducer.transformer(model = 'conformer-transducer-singlish', device = 'gpu:0')

2023-04-03 15:45:37.265516: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-03 15:45:37.272006: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-03 15:45:37.276604: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-03 15:45:37.278204: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [5]:
asr = malaya_speech.stt.transducer.pt_transformer(model = 'mesolitica/conformer-medium-mixed')
_ = asr.cuda()
_ = asr.eval()

In [6]:
lm = malaya.language_model.kenlm(model = 'bahasa-wiki-news-iium-stt')

In [7]:
lm.score('tak suka hg')

-13.193544387817383

In [8]:
import json
from glob import glob
from tqdm import tqdm

with open('/home/husein/ssd1/speech-bahasa/malay-asr-train.json') as fopen:
    ms = json.load(fopen)

In [9]:
with open('/home/husein/dev/malaya-speech/semisupervised-transducer.json') as fopen:
    semi = json.load(fopen)
    
filtered_X, filtered_Y = [], []
for i in range(len(semi['X'])):
    if semi['Y'][i][1] > 0.2 and len(semi['Y'][i][0]):
        filtered_X.append(semi['X'][i])
        filtered_Y.append(semi['Y'][i][0])
        
ms['X'] = ms['X'] + filtered_X
ms['Y'] = ms['Y'] + filtered_Y

In [10]:
with open('/home/husein/dev/malaya-speech/semisupervised-transducer-v2.json') as fopen:
    semi = json.load(fopen)
    
filtered_X, filtered_Y = [], []
for i in range(len(semi['X'])):
    if semi['Y'][i][1] > 0.2 and len(semi['Y'][i][0]):
        filtered_X.append(semi['X'][i])
        filtered_Y.append(semi['Y'][i][0])
        
ms['X'] = ms['X'] + filtered_X
ms['Y'] = ms['Y'] + filtered_Y

In [11]:
with open('/home/husein/malaya-speech/singlish-stt-train.json') as fopen:
    sg = json.load(fopen)

In [12]:
len(sg['X'])

3284901

In [13]:
sg['X'][:10]

['/home/husein/ssd2/imda/wav/5-75-tfrecord-1409.wav',
 '/home/husein/ssd2/imda/wav/5-118-tfrecord-1786.wav',
 '/home/husein/ssd2/imda/wav/4-77-tfrecord-2083.wav',
 '/home/husein/ssd2/imda/wav/2-39-tfrecord-78.wav',
 '/home/husein/ssd2/imda/wav/7-63-tfrecord-730.wav',
 '/home/husein/ssd2/imda/wav/5-68-tfrecord-1531.wav',
 '/home/husein/ssd2/imda/wav/0-17-tfrecord-3636.wav',
 '/home/husein/ssd2/imda/wav/1-113-tfrecord-1496.wav',
 '/home/husein/ssd2/imda/wav/7-108-tfrecord-1945.wav',
 '/home/husein/ssd2/imda/wav/5-63-tfrecord-3858.wav']

In [14]:
from datasets import Audio

sr = 16000
minimum = int(0.3 * sr)
audio = Audio(sampling_rate=sr)

In [15]:
def groupby(alignment, length, min_threshold = 0.3):
    r = []
    g = []
    for no, row in enumerate(alignment):
        
        if no > 0 and len(r) and alignment[no]['start'] - alignment[no-1]['end'] >= min_threshold:
            g.append(r)
            r = []
        
        r.append(row)

    return g

In [16]:
!ls

augmentation-switchboard-v15	    augment-switchboard-v15.json
augmentation-switchboard-v16	    augment-switchboard-v16.json
augmentation-switchboard-v17	    augment-switchboard-v2.ipynb
augment-switchboard-lm-Copy1.ipynb  augment-switchboard-v3.ipynb
augment-switchboard-lm-Copy2.ipynb  augment-switchboard-v4.ipynb
augment-switchboard-lm.ipynb	    augment-switchboard-v5.ipynb
augment-switchboard-lm-tts.ipynb    README.md
augment-switchboard-lm-v2.ipynb


In [17]:
directory = 'augmentation-switchboard-v18'
!mkdir {directory}

In [18]:
import torchaudio
import torch
from itertools import permutations

In [19]:
def calculate_wer(actual, hyp):
    """
    Calculate WER using `python-Levenshtein`.
    """
    import Levenshtein as Lev

    b = set(actual.split() + hyp.split())
    word2char = dict(zip(b, range(len(b))))

    w1 = [chr(word2char[w]) for w in actual.split()]
    w2 = [chr(word2char[w]) for w in hyp.split()]

    return Lev.distance(''.join(w1), ''.join(w2)) / len(actual.split())

In [20]:
data = {}
for O in tqdm(range(20000)):
    try:
        i_ms = sample_without_replacement(len(ms['X']), 4)
        i_sg = sample_without_replacement(len(sg['X']), 4)

        groups = []
        for i in i_ms:
            x = ms['X'][i]
            v = ms['Y'][i]
            y = audio.decode_example(audio.encode_example(x))['array']
            asr_ = asr.forward([y])[0]
            
            if asr_[0][1][0] < 0.15 and calculate_wer(v, asr_[0][0]) > 0.1:
                # print(f'skip ms {i}', v, asr_[0][0], asr_[0][1][0])
                continue
                
            result = model.predict(y, v)

            if ' '.join([r['text'] for r in result['words_alignment']]) != v:
                continue

            grouped = groupby(result['words_alignment'], len(y) / sr)
            # print(grouped)
            for g in grouped:
                y_ = y[int(g[0]['start'] * sr): int(g[-1]['end'] * sr) + minimum]
                if len(y_):
                    v_ = [g_['text'] for g_ in g]
                    groups.append((y_, v_))

        for i in i_sg:
            x = sg['X'][i]
            v = sg['Y'][i]
            y = audio.decode_example(audio.encode_example(x))['array']
            asr_ = asr.forward([y])[0]
            
            if asr_[0][1][0] < 0.15 and calculate_wer(v, asr_[0][0]) > 0.1:
                # print(f'skip sg {i}', v, asr_[0][0], asr_[0][1][0])
                continue
                
            result = singlish_model.predict(y, v)

            if ' '.join([r['text'] for r in result['words_alignment']]) != v:
                continue

            grouped = groupby(result['words_alignment'], len(y) / sr)
            for g in grouped:
                y_ = y[int(g[0]['start'] * sr): int(g[-1]['end'] * sr) + minimum]
                if len(y_):
                    v_ = [g_['text'] for g_ in g]
                    groups.append((y_, v_))
                    
        if not len(groups):
            continue
            
        groups = sorted(groups, key = lambda x: len(' '.join(x[1])), reverse = True)[:7]
        
        strings = [' '.join(g[1]) for g in groups]
        perm = list(permutations(range(len(groups))))
        
        scores = []
        for p in perm:
            s = ' '.join([strings[p_] for p_ in p])
            scores.append(lm.score(s))
            
        best_perm = perm[np.argmax(scores)]
        
        l = 0
        combine_y, combine_v = [], []
        index = 0
        while l < 15 and index < len(groups):
            g = groups[best_perm[index]]
            l_ = len(g[0]) / sr
            if l_ < 1.0:
                index += 1
                continue
            r_int = random.randint(3000, 16000)
            l += l_ + (r_int / sr)
            combine_y.append(g[0] / np.abs(g[0]).max())
            combine_y.append(np.zeros(shape = (r_int,)))
            combine_v.extend(g[1])
            index += 1

        if len(combine_v):
            audio_path = f'{directory}/{O}.mp3'
            torchaudio.save(audio_path, 
                            torch.tensor(np.concatenate(combine_y).astype('float32')).unsqueeze(0), 
                            sr, format='mp3')
            data[O] = ' '.join(combine_v)
            
    except Exception as e:
        pass

  0%|                                                                                                          | 0/20000 [00:00<?, ?it/s]2023-04-03 15:46:26.003702: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-04-03 15:46:26.003739: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-04-03 15:46:26.003763: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-04-03 15:46:26.009184: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8302
2023-04-03 15:46:26.010300: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
 24%|█████████████████████▊                                                                    | 4853/2

KeyboardInterrupt: 

In [21]:
len(data)

4436

In [22]:
data

{0: 'saranan ataupun',
 1: 'begitulah juga kes kebenaran kita nak sebarkan kebenaran warganegara asing sepanjang tempoh perintah kawalan pergerakan macam kita sebut tadilah sebagai seorang manusia biasa',
 2: 'kepada kita itulah dia tuan tuan wykidd song by using names such as foo ah bee and lee peh gee duk tang mana roh tentang roh ni duk tang mana',
 3: 'sabar sabar semua kalau satu komponen tu yang bukan tempat dia tuan tuan topik utama ialah kami membincangkan pelbagai jenis fitnah dan rasulullah turut menyertai kami berbicara tentang beberapa jenis fitnah',
 4: 'the cashback credit card and date of birth is current promotion if i were to twenty nine november',
 5: 'facebook added that it is continuing to see an increase in content restrictions and government requests segala galanya hitam',
 7: 'yang tu dah jadi lain dah but you must produce us document relevant document anak bebas menggunakan internet setiap masa',
 8: 'karley and conflict is resolved',
 9: 'kata semua orang lain 

In [23]:
import IPython.display as ipd
ipd.Audio(f'{directory}/3.mp3')

In [24]:
directory

'augmentation-switchboard-v18'

In [25]:
with open('augment-switchboard-v18.json', 'w') as fopen:
    json.dump(data, fopen)